In [1]:
from pathlib import Path

# This should match your project root
BASE_DIR = Path.cwd().parent.parent.parent

DATA_RAW = BASE_DIR / "data" / "raw"
DATA_PROCESSED = BASE_DIR / "data" / "processed"

print("BASE_DIR:", BASE_DIR)
print("DATA_RAW exists:", DATA_RAW.exists())
print("DATA_PROCESSED exists:", DATA_PROCESSED.exists())

BASE_DIR: /files/capstone_project/game-market-event-analyzer
DATA_RAW exists: True
DATA_PROCESSED exists: True


In [2]:
import pandas as pd
from pathlib import Path

events = pd.read_csv(DATA_PROCESSED / "events_with_car.csv")
events["trading_date"] = pd.to_datetime(events["trading_date"])
events["event_date"] = pd.to_datetime(events["event_date"])

events.head()


,event_id,event_date,trading_date,ticker,is_rockstar,event_type,sentiment,impact_expectation_manual,adj_close,return,market_return,AR_event,CAR_m1_p1,CAR_m5_p5
0,ATVI_2019_CODMOBILE_LAUNCH,2019-10-01,2019-10-01,ATVI,0,Game_release,Positive,Medium,94.157463,-0.010938,-0.012258,-0.001198,0.005278,-0.013979
1,ATVI_2019_CODMW_RELEASE,2019-10-25,2019-10-25,ATVI,0,Game_release,Positive,High,93.729248,0.003438,0.004073,-0.000318,0.000526,-0.002328
2,ATVI_2020_WARCRAFT3_REFORGED,2020-01-28,2020-01-28,ATVI,0,Bad_review,Negative,Medium,108.901489,0.012120,0.010054,0.003422,0.003371,-0.024705
3,ATVI_2020_WARZONE_LAUNCH,2020-03-10,2020-03-10,ATVI,0,Game_release,Positive,High,100.609787,0.024274,0.049396,-0.016937,0.001569,-0.031189
4,ATVI_2021_LAWSUIT,2021-07-20,2021-07-20,ATVI,0,Lawsuit,negative,high,137.807159,-0.001204,0.015163,-0.014124,-0.024080,0.003237


In [3]:
prices = pd.read_csv(DATA_PROCESSED / "prices_with_ar.csv")
prices["date"] = pd.to_datetime(prices["date"])

In [4]:
def compute_car(prices, ticker, trading_date, start, end):
    mask = (
        (prices["ticker"] == ticker) &
        (prices["date"] >= trading_date + pd.Timedelta(days=start)) &
        (prices["date"] <= trading_date + pd.Timedelta(days=end))
    )
    return prices.loc[mask, "AR"].sum()

In [5]:
events["CAR_0_1"] = events.apply(
    lambda r: compute_car(prices, r["ticker"], r["trading_date"], 0, 1),
    axis=1
)

events["CAR_0_3"] = events.apply(
    lambda r: compute_car(prices, r["ticker"], r["trading_date"], 0, 3),
    axis=1
)

events["CAR_0_5"] = events.apply(
    lambda r: compute_car(prices, r["ticker"], r["trading_date"], 0, 5),
    axis=1
)

In [6]:
events["CAR_m1_p1"]  # already computed earlier

0     0.005278
1     0.000526
2     0.003371
3     0.001569
4    -0.024080
        ...   
67    0.001016
68    0.018096
69    0.015610
70    0.047224
71    0.003606
Name: CAR_m1_p1, Length: 72, dtype: float64

In [7]:
# Define ML labels
def label_impact(car):
    if abs(car) > 0.03:
        return "High"
    elif abs(car) > 0.01:
        return "Medium"
    else:
        return "Low"

In [8]:
events["impact_label"] = events["CAR_m1_p1"].apply(label_impact)
events["impact_label"].value_counts()

impact_label
Low       28
High      27
Medium    17
Name: count, dtype: int64

In [9]:
out = DATA_PROCESSED / "events_labeled.csv"
events.to_csv(out, index=False)
out

PosixPath('/files/capstone_project/game-market-event-analyzer/data/processed/events_labeled.csv')